# Support Vector Machine with Polynomial and RBF Kernel (Random Search)

In [1]:
# Increase the width of the notebook
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import uniform, randint
import joblib
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

y_train = train["Score"]
y_test = test["Score"]

X_train = train.drop("Score", axis=1)
X_test = test.drop("Score", axis=1)

In [3]:
# Preprocessing setup
numeric_features = ["WhiteElo", "EloDif"]
categorical_features = ["Opening_name", "Time_format", "Increment_binary"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

## Randomized Search for combined kernel SVC

pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("svc", SVC())
])

param_dist = {
    "svc__kernel": ["rbf", "poly"],
    "svc__C": uniform(0.1, 10),
    "svc__gamma": ["scale", "auto"],
    "svc__degree": randint(2, 5),
    "svc__class_weight": [None, "balanced"]
}

random_search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=20,
    cv=3,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train, y_train)
print("Best params (random search):", random_search.best_params_)
print("Test set score:", random_search.score(X_test, y_test))

##  RBF SVC (Randomized Search) 

In [4]:
rbf_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("svc", SVC(kernel="rbf", random_state=42))
])

param_dist_rbf = {
    "svc__C": uniform(0.1, 10),
    "svc__class_weight": [None, "balanced"]
}

random_rbf = RandomizedSearchCV(
    rbf_pipe,
    param_distributions=param_dist_rbf,
    n_iter=10,
    cv=3,
    scoring="accuracy",
    random_state=42,
    n_jobs=1
)
random_rbf.fit(X_train, y_train)
print("RBF best params:", random_rbf.best_params_)
print("RBF test accuracy: {:.3f}".format(random_rbf.score(X_test, y_test)))

RBF best params: {'svc__C': 3.845401188473625, 'svc__class_weight': None}
RBF test accuracy: 0.541


In [5]:
best_svc_rbf = random_rbf.best_estimator_
joblib.dump(best_svc_rbf, 'best_svc_rbf.joblib')

['best_svc_rbf.joblib']

# Polynomial SVC (Randomized Search) 

In [7]:
poly_pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("svc", SVC(kernel="poly", random_state=42))
])

param_dist_poly = {
    "svc__C": uniform(0.1, 10),
    "svc__degree": randint(2, 4),
    "svc__class_weight": [None, "balanced"]
}

random_poly = RandomizedSearchCV(
    poly_pipe,
    param_distributions=param_dist_poly,
    n_iter=10,
    cv=3,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)
random_poly.fit(X_train, y_train)
print("Poly best params:", random_poly.best_params_)
print("Poly test accuracy: {:.3f}".format(random_poly.score(X_test, y_test)))

best_svc_poly = random_poly.best_estimator_
joblib.dump(best_svc_poly, 'best_svc_poly.joblib')

Poly best params: {'svc__C': 1.9182496720710063, 'svc__class_weight': None, 'svc__degree': 2}
Poly test accuracy: 0.535


['best_svc_poly.joblib']